# Import Modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 KB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai: filename=openai-0.26.0-py3-none-any.whl size=66855 sha256=a4c81fc01dd0f71f7016abfd4ce0cc6bfb918100e11d5f4d49a2d39337261437
  Stored in directory: /root/.cache/pip/wheels/7e/4c/c8/31e9d441bd937e2b9076627465f9db43ab6db40f08aae60b66
Successfully built openai


In [ ]:
import numpy as np
import os
import openai
import pandas as pd
import time
import traceback
from openai.embeddings_utils import get_embedding
from tqdm import tqdm

In [ ]:
%cd "/content/drive/MyDrive/course_selection"

/content/drive/MyDrive/course_selection


# Copy and paste your OpenAI API Key below.

In [ ]:
openai.api_key = "" #@param {type:"string"}

To obtain a key, go to `https://beta.openai.com/account/api-keys` and create an API key after making an OpenAI account.

# Main Code

In [ ]:
CHUNK_SIZE=2000
df = pd.read_csv("mit_professor_data_raw.csv")

In [ ]:
df['embedding_school'] = [[] for _ in range(len(df))]
df['embedding_desc'] = [[] for _ in range(len(df))]
df['embedding_combined'] = [[] for _ in range(len(df))]
df['combined'] = "Title: " + df.name.str.strip() + "; Content: " + df.keywords.str.strip()
df.fillna('none', inplace=True) # fill all NaN's in the dataframe

# df_keys = ["course_number","title","hours","cluster","desc","terms","optional","instructors","prereq","embedding_title", "embedding_desc", "embedding_combined"]

pbar = tqdm(total=len(df))
for i in range(int(np.ceil(len(df)/CHUNK_SIZE))):
    start = i * CHUNK_SIZE
    end = start + CHUNK_SIZE
    end = min(end, len(df))
    model_school = openai.Embedding.create(model="text-embedding-ada-002", input=list(df['school_affiliation'][start:end]))
    model_desc = openai.Embedding.create(model="text-embedding-ada-002", input=list(df['keywords'][start:end]))
    model_combined = openai.Embedding.create(model="text-embedding-ada-002", input=list(df['combined'][start:end]))
    for j in range(end-start):
        embedding_school = model_school.data[j]['embedding']
        df.at[start + j, 'embedding_school'] = embedding_school
        embedding_desc = model_desc.data[j]['embedding']
        df.at[start + j, 'embedding_desc'] = embedding_desc
        embedding_combined = model_combined.data[j]['embedding']
        df.at[start + j, 'embedding_combined'] = embedding_combined
        pbar.update()

df.to_csv("mit_professors_with_embeddings.csv", index=False)
print("\n\nComplete!")


  0%|          | 0/2289 [01:13<?, ?it/s]

 87%|████████▋ | 2001/2289 [00:11<00:01, 228.98it/s]



Complete!
